In [15]:
import struct

def rotate_left(x, bits, width=64):
    return ((x << bits) | (x >> (width - bits))) & ((1 << width) - 1)

# Toy compression function: XOR and rotate
def compression(h, m):
    return rotate_left(h ^ m, 7)

# Pad message using Merkle–Damgård strengthening
def pad_message(msg_bytes, block_size=8):
    msg_len_bits = len(msg_bytes) * 8
    # Append '1' bit then zeroes
    padded = msg_bytes + b'\x80'
    while (len(padded) + 8) % block_size != 0:
        padded += b'\x00'
    # Append 64-bit length
    padded += struct.pack('>Q', msg_len_bits)
    return padded

def merkle_damgard_hash(message: bytes, iv=0x0123456789ABCDEF, block_size=8):
    padded = pad_message(message, block_size)
    h = iv
    for i in range(0, len(padded), block_size):
        block = padded[i:i + block_size]
        m = int.from_bytes(block, 'big')
        h = compression(h, m)
    return h

def hash_to_hex(h: int) -> str:
    return hex(h)[2:].zfill(16)

# Example usage
msg = b"A un amigo perdido. Tu ya no debes recordarme, en el pais extranjero."
digest = merkle_damgard_hash(msg)
print("Hash:", hash_to_hex(digest))


Hash: 9595b678e83c1071
